# The Many Ways To Apply A Multi-Step Conditional Function to a Pandas Series
- **Sources**
    - (12 Ways to Apply a Function to Each Row in Pandas DataFrame by Satish Chandra Gupta (Medium))[https://towardsdatascience.com/apply-function-to-pandas-dataframe-rows-76df74165ee4]
    - (How To USe Pandas the Right way to speed up your code by George Seif (Medium)[https://towardsdatascience.com/how-to-use-pandas-the-right-way-to-speed-up-your-code-4a19bd89926d]
    - (How I customarily bin data with Pandas by Bex T (Medium)[https://towardsdatascience.com/how-i-customarily-bin-data-with-pandas-9303c9e4d946]

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset("diamonds")

### Explore data

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [7]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [9]:
df.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


### Task
- Create a new column that categorises by price
    - Categories: cheap, pricey, expensive, super_expensive
     - Ranges: <500, 500-2500, 2500-10000, >10000

### Define function

In [4]:
def classify_by_price(price):
    if price < 500:
        return "cheap"
    elif 500 <= price < 2500:
        return "pricey"
    elif 2500 <= price < 10000:
        return "expensive"
    else:
        return "super_expensive"

### Establish a base line

#### Pure Python for loop
- **Runtime = 7170 ms**

In [15]:
%%timeit
class_list_loop = []
for i in range(len(df)):
    price = df.iloc[i]["price"]
    class_list_loop.append(classify_by_price(price))

df["price_cat_loop"] = class_list_loop

7.17 s ± 267 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Pure Python for loop replace .iloc with .at
- **Runtime = 252 ms**

In [24]:
%%timeit
class_list_loop_at = []
for i in range(len(df)):
    price = df.at[i, "price"]
    class_list_loop_at.append(classify_by_price(price))

df["price_cat_loop_at"] = class_list_loop_at

252 ms ± 6.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Use iterrows()
- **Runtime = 1960 ms**

In [26]:
%%timeit
class_list_iterrow = []
for _, row in df.iterrows():
    price = row["price"]
    class_list_iterrow.append(classify_by_price(price))

df["price_cat_iterrow"] = class_list_iterrow

1.96 s ± 137 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Use itertuples()
- **Runtime = 77.6 ms**

In [7]:
%%timeit
class_list_itertuple = []
for tup in df.itertuples():
    price = tup.price
    class_list_itertuple.append(classify_by_price(price))

df["price_cat_itertuple"] = class_list_itertuple

### Use apply()
- **Runtime = 386 ms**

In [36]:
%timeit df["price_cat_apply"] = df.apply(lambda row: classify_by_price(row["price"]), axis=1)

386 ms ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Use python list comprehension
- **Runtime= 14.0 ms**

In [12]:
%timeit df["price_cat_list_comp"] = [classify_by_price(x) for x in df["price"]]

14 ms ± 450 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Use pandas map function
- **Runtime = 12.8 ms**

In [11]:
%timeit df["price_cat_map"] = df["price"].map(classify_by_price)

12.8 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Use Pandas Cut function
- **Runtime = 1.96 ms**

In [22]:
%%timeit
df["price_cat_cut"] = pd.cut(x=df["price"],
                            bins=[0, 500, 2500, 10000, 20000],
                            include_lowest=True,
                            labels=["cheap", "pricey", "expensive", "super_expensive"])

1.96 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Use numpy.vectorize()
- **Runtime = 26.7 ms**

In [18]:
%%timeit
vfunc = np.vectorize(classify_by_price)
df["price_cat_npvec"] = vfunc(df["price"])

26.7 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Use numpy.select()
- **Runtime = 10.6 ms**

In [21]:
%%timeit
conditions = [
    df["price"] < 500,
    df["price"].between(500, 2500, inclusive="left"),
    df["price"].between(2500, 10000, inclusive="left")
]

choices = [
    "cheap",
    "pricey",
    "expensive"
]

df["price_cat_npselect"] = np.select(conditions, choices, default="super_expensive")

10.6 ms ± 209 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Conclusion

## Summary of methods and timings
For a dataframe with 53,940 rows:

In [35]:
timings = {
    "Method": ["Python For Loop (.iloc)", "Python for loop (.at)", "Loop with iterrows",
               "Loop with itertuples", "Pandas apply", "Python list comprehension",
               "Pandas map", "Pandas cut", "Numpy vectorise", "Numpy select"],
    "Time(ms)": [7170.0, 252.0, 1960.0, 77.6, 386, 14.0, 12.8, 1.96, 26.7, 10.6]
}
results = pd.DataFrame(timings)

In [55]:
results.sort_values("Time(ms)", ascending=False)

,Method,Time(ms)
0,Python For Loop (.iloc),7170.00
2,Loop with iterrows,1960.00
4,Pandas apply,386.00
1,Python for loop (.at),252.00
3,Loop with itertuples,77.60
8,Numpy vectorise,26.70
5,Python list comprehension,14.00
6,Pandas map,12.80
9,Numpy select,10.60
7,Pandas cut,1.96


In this case pandas.cut() was the fastest method, however I have done similar test on slightly smaller datasets e.g. 'iris' and both numpy methods were faster.

**Takeaway**: There are many ways to do the same thing in pandas, no single method is guaranteed to be the fastest each time. If performance is of concern you will have to profile to see which is the best.